In [ ]:
CHILD_LOG_FILE="/Users/law/repos/ma/benchmark-runs/micro_measure_child/micro_measure.log"
ROOT_LOG_FILE="/Users/law/repos/ma/benchmark-runs/micro_measure_root/micro_measure_root.log"

In [ ]:
import re
from collections import defaultdict

RECV_LINE = re.compile(r"Avg receiving  time: (\d+) ns.")
PROC_TIME = re.compile(r"Avg processing time: (\d+) ns.")
WM_PROC_LINE = re.compile(r"Watermark processing took (\d+) ns.")
WM_SEND_LINE = re.compile(r"Watermark sending took (\d+) ns.")
SESSION_LINE = re.compile(r"Session start took (\d+) ns.")
LOWER_TIME = re.compile(r"Avg lower time: (\d+) ns.")
RES_SENDING_TIME = re.compile(r"Avg sending time: (\d+) ns.")
MAX_LINE = "TUMBLING,1000 MAX"
AVG_LINE = "TUMBLING,1000 M_AVG"
MEDIAN_LINE = "TUMBLING,1000 M_MEDIAN"

FUNCTIONS = ["median", "max", "avg"]

CHILD_MATCHERS = [
    (RECV_LINE, "recv"), (PROC_TIME, "proc"), 
    (WM_PROC_LINE, "wm-proc"), (WM_SEND_LINE, "wm-send"),
    (SESSION_LINE, "session"),
]

ROOT_MATCHERS = [
    (RECV_LINE, "recv"), (PROC_TIME, "proc"),
    (LOWER_TIME, "lower"), (RES_SENDING_TIME, "sending")
]

def get_processing_times(log_file, is_child=True):
    matchers = CHILD_MATCHERS if is_child else ROOT_MATCHERS 
    
    with open(log_file) as f:
        times = {}
        for fn in FUNCTIONS:
            times[fn] = {}
            for _, key in matchers:
                times[fn][key] = []

        current_function = None
        for line in f:
            if MAX_LINE in line:
                current_function = "max"
            if AVG_LINE in line:
                current_function = "avg"
            if MEDIAN_LINE in line:
                current_function = "median"
                
            for matcher, key in matchers:
                match = matcher.match(line)
                if match is not None:
                    duration = int(match.group(1))
                    times[current_function][key].append(duration)
    return times


ALL_TIMES = get_processing_times(CHILD_LOG_FILE)
# print(ALL_TIMES)

ROOT_TIMES = get_processing_times(ROOT_LOG_FILE, is_child=False)
print(ROOT_TIMES)

In [ ]:
def clean_all_times(all_times):
    clean_times = {}
    for fn, proc_times in all_times.items():
        clean_times[fn] = {}
        for key, times in proc_times.items():
            num_times = len(times)
            cut_off = max(1, int(num_times * 0.05))
            cut_times = times[cut_off:-cut_off] 
            avg = sum(cut_times) / len(cut_times)
            print(f"{fn} - {key}: {avg:.2f} ns ({num_times} values)")
            clean_times[fn][key] = cut_times
            clean_times[fn][f"avg-{key}"] = avg
    return clean_times


print("Child node\n==========")
CLEAN_TIMES = clean_all_times(ALL_TIMES)
for fn, proc_times in CLEAN_TIMES.items():
    recv = proc_times['avg-recv']
    proc = proc_times['avg-proc']
    avg_both = recv + proc
    one_sec_ns = 1_000_000_000
    max_per_sec = one_sec_ns / avg_both
    # We measure 33ns duration for each call to System.nanoTime(), so we add it to second
    syscall_ns = 33 * 8
    clean_sec_ns = one_sec_ns + (max_per_sec * syscall_ns)
    max_per_sec_clean = clean_sec_ns / avg_both
    print(f"{fn}: {max_per_sec:.0f} events/s.")
    print(f"{fn}: {max_per_sec_clean:.0f} clean events/s.")

print("\nRoot node\n=========")
CLEAN_ROOT_TIMES = clean_all_times(ROOT_TIMES)
for fn, proc_times in CLEAN_ROOT_TIMES.items():
    recv = proc_times['avg-recv']
    proc = proc_times['avg-proc']
    lower = proc_times['avg-lower']
    avg_both = recv + proc + lower
    one_sec_ns = 1_000_000_000
    max_per_sec = one_sec_ns / avg_both
    # We measure 33ns duration for each call to System.nanoTime(), so we add it to second
    syscall_ns = 33 * 8
    clean_sec_ns = one_sec_ns + (max_per_sec * syscall_ns)
    max_per_sec_clean = clean_sec_ns / avg_both
    print(f"{fn}: {max_per_sec:.0f} events/s.")
    print(f"{fn}: {max_per_sec_clean:.0f} clean events/s.")       

In [ ]:
from matplotlib import rcParams
import numpy as np
import matplotlib.pyplot as plt
rcParams.update({'figure.autolayout': True, 'pgf.rcfonts' : False, 'font.size': 16, 'lines.linewidth': 3})
plt.style.use('seaborn-deep')

In [ ]:
def plot_run_times(times):
    fig, ax = plt.subplots()
    fig.set_tight_layout(False)
    
    bar_width = 0.20
    x_locations = [0, 1, 2, 3]
    
    for i, fn in enumerate(['max', 'avg', 'median']):
        fn_times = times[fn]
        median_op_durations = []
        stddev_op_durations = []
        
        recv_times = times['max']['recv'][30:90]
        recv_op_time = np.median(recv_times)
        stddev_recv_op_time = np.std(recv_times)
        median_op_durations.append(recv_op_time)
        stddev_op_durations.append(stddev_recv_op_time)
        
        for operation in ('proc', 'wm-proc', 'wm-send'):
            op_times = fn_times[operation][30:90]
            median_op_time = np.median(op_times)
            median_op_durations.append(median_op_time)
            stddev_op_time = np.std(op_times)
            stddev_op_durations.append(stddev_op_time)
            
        ith_x_locations = [x + (i*bar_width) for x in x_locations]
        ax.bar(ith_x_locations, median_op_durations, bar_width, bottom=0, label=fn, yerr=stddev_op_durations) 
        
    ax.set_ylabel("operation duration in ns")
    ax.set_xticks([x + bar_width for x in x_locations])
    ax.set_xticklabels(('receive\nevent', 'process\nevent', 'process\nwatermark', 'send\nresult'))
    plt.legend()
    plt.yscale('symlog')
#     plt.savefig("/tmp/plots/child-micro.png", bbox_inches="tight")
#     plt.savefig("/tmp/plots/child-micro.pdf", bbox_inches="tight")
    plt.show()
    
plot_run_times(CLEAN_TIMES)

In [ ]:
def plot_proc_times(fn, proc_times):
    entries = sorted(proc_times.items())
    keys, times = zip(*entries)
    N = len(keys)
    plt.boxplot(times, showfliers=False)
    plt.xticks(range(1, len(keys) + 1), keys)
    plt.yscale('log')
    plt.title(fn)
    plt.show()

for fn in FUNCTIONS:
    plot_proc_times(fn, ALL_TIMES[fn])